In [1]:
import re
import os

import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
# ----------------------------------------------- #
import tensorflow as tf
from tensorflow.keras import backend  as K
from tensorflow.keras.models import Model
from tensorflow.keras import Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, TimeDistributed, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, pad_sequences
from sklearn.model_selection import train_test_split


lothar = '/home/enno/uni/SS24/thesis/1_seq_analysis'

2024-05-06 14:40:33.195561: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 14:40:33.201969: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-06 14:40:33.268760: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-06 14:40:35.625115: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
df = pd.read_csv('/ebio/abt1_share/prediction_hendecads/1_repo/2_ss_pred/dataset/annotated_df.csv')
df.head()

,id,seq,stretch_ix,stretch_seq,avg_cc,group_id,group_ix,1R_hendecad_loose,1R_ratio_hendecad_loose,1R_ix_hendecad_loose,...,10R_ix_hendecad_alanine,10R_heptad,10R_ratio_heptad,10R_ix_heptad,y,x,_,group,reduced_group,annotation
0,MCD6041253.1,Mrlvyvavaailcsfsttslagaektakragkfvektatragkfve...,"[1, 71]",rlvyvavaailcsfsttslagaektakragkfvektatragkfver...,0.001041,unassigned,0,11,0.183333,"[(1, 12), (2, 13), (18, 29), (21, 32), (25, 36...",...,[],0,0.0,[],-567.38850,34.030270,0.0,0,Other,Other
1,MCD7737945.1,MQGRVFFREAAALILAAALSMAGLPASAAANSGIEAAALRTeeete...,"[41, 238]",eeetepstkeavqetavetdtgekpesgedgqeesaesteeeqeed...,0.000364,unassigned,0,1,0.005348,"[(50, 61)]",...,[],0,0.0,[],313.18863,-218.576080,0.0,0,Other,Other
2,MYF28459.1,Merlqtdllkeihalrgemhaefasvrqemhagfasirqemhaeta...,"[1, 73]",erlqtdllkeihalrgemhaefasvrqemhagfasirqemhaetas...,0.001002,unassigned,0,10,0.161290,"[(6, 17), (10, 21), (12, 23), (17, 28), (28, 3...",...,[],0,0.0,[],535.79120,-22.468689,0.0,0,Other,Other
3,WP_168920948.1,MSDVFLTASYADREKVKTLGARWNPAEKRWYVPSGRDLSPFAAWLP...,"[437, 540]",aqslvveikhaasqqlllarhvvparmaevtaegrqalrtakaqsq...,0.000689,group_42,1,17,0.182796,"[(0, 11), (3, 14), (4, 15), (10, 21), (11, 22)...",...,[],0,0.0,[],-346.27220,104.598206,0.0,1,Other,Other
4,WP_026306873.1,MLLRRIARPLLSAAFIAEGIDILQNPGPLADRLSPALDFTRRRSQH...,"[172, 342]",slgwrgrraardakdhaealaataaaiaatarergtnlvdtarert...,0.000424,group_38,2,29,0.181250,"[(1, 12), (9, 20), (12, 23), (16, 27), (18, 29...",...,[],0,0.0,[],164.44267,341.078280,0.0,2,Other,Other


In [9]:
for col in [col for col in df.columns if 'strict' in col][::3]:
    print(col)
    print(df[col].value_counts())
    print('')

1R_hendecad_strict
1R_hendecad_strict
0      6744
1      2883
2      2618
3      2409
4      2348
       ... 
119       1
282       1
158       1
185       1
184       1
Name: count, Length: 177, dtype: int64

2R_hendecad_strict
2R_hendecad_strict
0      21441
1       4118
2       2722
3       2143
4       1617
       ...  
207        1
98         1
199        1
92         1
104        1
Name: count, Length: 121, dtype: int64

3R_hendecad_strict
3R_hendecad_strict
0      28994
1       2969
2       1991
3       1333
4        975
       ...  
108        1
156        1
95         1
162        1
89         1
Name: count, Length: 101, dtype: int64

4R_hendecad_strict
4R_hendecad_strict
0      33197
1       2089
2       1309
3        892
4        590
       ...  
98         1
130        1
67         1
91         1
79         1
Name: count, Length: 92, dtype: int64

5R_hendecad_strict
5R_hendecad_strict
0      35745
1       1388
2        875
3        571
4        355
       ...  
141        1

In [ ]:
# write 3R hits to fasta to process in CLANS, CD-Hit, etc.

def regex_to_fasta(col):
    if os.getcwd() == lothar:
        fasta = open(f'/home/enno/uni/SS24/thesis/data/results/seq_analysis/tmp/fasta/{col}_tmp.fasta', 'w')
    else:
        fasta = open(f'/ebio/abt1_share/prediction_hendecads/0_data/1_seq_analysis/tmp/fasta/{col}_tmp.fasta', 'w')
    col_ix = col[:3] + 'ix_' + col[3:]

    for i in range(len(df)):
        current_seq = df.iloc[i]

        if len(current_seq[col_ix]) == 0:
            continue

        else:
            for ix in eval(current_seq[col_ix]):
                print(ix)
                # print(f'>{current_seq["id"]}_[{ix[0]}, {ix[1]}]')

                # seq = current_seq['stretch_seq'][ix[0]-11:ix[1]+11].upper()
                # print('a--d---h---')
                # for i in range(int(col[0])+2):
                #     print(string_to_color(seq[i*11:(i+1)*11]))

                fasta.write(f'>{current_seq["id"]}_[{ix[0]},{ix[1]}]\n')
                seq = current_seq['stretch_seq'][ix[0]:ix[1]].upper()
                fasta.write(seq + '\n')

    fasta.close()

regex_to_fasta('3R_hendecad_strict')

In [ ]:
# cd 3_CLANS/
# ./1_blast_all_vs_all.sh 
# ssh dol
# qsub -N clans_25k_3R_tmp -pe parallel 32 -cwd -sync n -now n -m beas -o tmp/logs/clans/ -j y /ebio/abt1_share/prediction_hendecads/3_CLANS/run_clans.sh -l v_mem=16G 

In [23]:
def build_model(n_words, max_seq_len=1024):
    input = Input(shape=(max_seq_len,))
    
    mbed = Embedding(input_dim=n_words, output_dim=128)(input)
    comp = Bidirectional(LSTM(units=64, return_sequences=False, recurrent_dropout=0.1))(mbed)
    
    final_comp = Dense(1, activation="sigmoid")(comp)
    
    model = Model(input, final_comp)
    
    return model


In [32]:
def tokenize_data(df, 
                  max_seq_len=1024, data_flavor='kaggel'):
    
    tokenizer_in = Tokenizer(char_level=True)
    tokenizer_in.fit_on_texts(df.stretch_seq)
    
    # tokenizer_out = Tokenizer(char_level=True)
    
    # if data_flavor == 'kaggel':
    #     tokenizer_out.fit_on_texts(df.sst3)
    #     y = tokenizer_out.texts_to_sequences(df.sst3)    
            
    X = tokenizer_in.texts_to_sequences(df.stretch_seq)
    X = pad_sequences(X, maxlen=max_seq_len, padding='post')

    # y = pad_sequences(y, maxlen=max_seq_len, padding='post')
    # y = to_categorical(y)

    n_words = len(tokenizer_in.word_index) + 1
    # n_tags = len(tokenizer_out.word_index) + 1

    return X, n_words


In [43]:
def split_data(X, y, 
               test_size=0.2, val_size=0.5, random_state=42):
    
    X_train, X_, y_train, y_ = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=val_size, random_state=random_state, stratify=y_, shuffle=True)
    
    return X_train, X_val, X_test, y_train, y_val, y_test

In [47]:
def train_model(model, X_train, X_val, y_train, y_val, 
                batch_size=32, epochs=5):
 
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy", "AUC"])
    
    model.fit(X_train, y_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(X_val, y_val), 
              verbose=1)
    model.t
    return model

### Model

In [44]:
X, n_words = tokenize_data(df)

In [45]:
y = df["3R_hendecad_strict"] > 0

In [46]:
X_train, X_val, X_test, y_train, y_val, y_test = split_data(X, y)

In [50]:
len(X_train), len(X_val), len(X_test)

(32472, 4059, 4059)

In [51]:
sum(y_train) / len(y_train), sum(y_val) / len(y_val), sum(y_test) / len(y_test)

(0.2856922887410692, 0.2857846760285785, 0.28553830992855384)

In [40]:
model = build_model(n_words)

2024-05-06 14:31:04.917071: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


In [48]:
model = train_model(model, X_train, X_val, y_train, y_val)

Epoch 1/5
  55/1015 ━━━━━━━━━━━━━━━━━━━━ 11:19 707ms/step - AUC: 0.5552 - accuracy: 0.6626 - loss: 0.6166

KeyboardInterrupt: 